<h1>NB model on kaggle-pet competition</h1>

<h1>DATA LOADING

In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
os.listdir('../../data')
assert 'out_breed.csv' in os.listdir('../../data') # this assert breaks if the data is configured uncorrectly

In [3]:
breeds = pd.read_csv('../../data/out_breed.csv')
colors = pd.read_csv('../../data/out_color.csv')
states = pd.read_csv('../../data/out_state.csv')
train  = pd.read_csv('../../data/out_train.csv')
test   = pd.read_csv('../../data/out_test.csv')
sub    = pd.read_csv('../../data/out_submission.csv')

<h1>MODEL</h1>

In [12]:
from gaussianNaiveBayes import PredictiveModel as gaussianPredictiveModel
from multinomialNaiveBayes import PredictiveModel as multinomialPredictiveModel

<h1>EXAMPLE USAGE</h1>

In [5]:
string_cols = ["Unnamed: 0", "dataset_type", "Name", "RescuerID", "Description", "PhotoAmt","VideoAmt","PetID"]
categorical_col = ["Type","Gender","Vaccinated","Dewormed","Sterilized","Breed1","Breed2","Color1","Color2","Color3","State"]
numerical_col = [col for col in train.columns if col not in string_cols and col not in categorical_col and col != "AdoptionSpeed"]

numerical_X = train[numerical_col]
numerical_X_test = train[numerical_col]

X = train.drop(["AdoptionSpeed", "Unnamed: 0", "dataset_type", "Name", "RescuerID", "Description", "PhotoAmt","VideoAmt","PetID"], axis=1)
X_test = test.drop(["Unnamed: 0", "dataset_type", "Name", "RescuerID", "Description", "PhotoAmt","VideoAmt","PetID"], axis=1)

"""
Y is our target value, Adoption Speed can be a value [1,2,3,4]
"""
Y = train['AdoptionSpeed']

assert X.shape[0] == Y.shape[0]

In [7]:
model = gaussianPredictiveModel("gNB")
model.train(numerical_X, Y, verbose=True)
predictions = model.predict(numerical_X_test)
assert len(predictions)

Wed Mar  6 03:19:20 2019 [gNB.__init__] initialized succesfully
Wed Mar  6 03:19:20 2019 [gNB.train] start training
Wed Mar  6 03:19:20 2019 [gNB.train] WARNING: Index(['Age', 'MaturitySize', 'FurLength', 'Health', 'Quantity', 'Fee'], dtype='object') must be Gaussian
Wed Mar  6 03:19:20 2019 [gNB.train] trained succefully


<h1>VALIDATION

In [18]:
string_cols = ["Unnamed: 0", "dataset_type", "Name", "RescuerID", "Description", "PhotoAmt","VideoAmt","PetID"]
categorical_col = ["Type","Gender","Vaccinated","Dewormed","Sterilized","Breed1","Breed2","Color1","Color2","Color3","State"]
numerical_col = [col for col in train.columns if col not in string_cols and col not in categorical_col and col != "AdoptionSpeed"]

numerical_X = train[numerical_col]
numerical_X_test = train[numerical_col]

X = train.drop(["AdoptionSpeed", "Unnamed: 0", "dataset_type", "Name", "RescuerID", "Description", "PhotoAmt","VideoAmt","PetID"], axis=1)
X_test = test.drop(["Unnamed: 0", "dataset_type", "Name", "RescuerID", "Description", "PhotoAmt","VideoAmt","PetID"], axis=1)

"""
Y is our target value, Adoption Speed can be a value [1,2,3,4]
"""
Y = train['AdoptionSpeed']

assert X.shape[0] == Y.shape[0]

gaussianPredictiveModel validation

In [19]:
model = gaussianPredictiveModel("gNB_validation_model")

Wed Mar  6 04:05:52 2019 [gNB_validation_model.__init__] initialized succesfully


In [20]:
model.validation(numerical_X, Y, method=1)

0.049338037187933906

In [21]:
model.validation(numerical_X, Y, method=2)

0.051837035594567246

In [22]:
categorical_X=X['Breed1']
mapping_size=len(breeds)

multinomialPredictiveModel validation

In [23]:
model = multinomialPredictiveModel("mNB_validation_model")

Wed Mar  6 04:05:57 2019 [mNB_validation_model.__init__] initialized succesfully


In [24]:
model.validation(categorical_X, Y, mapping_size, method=1)

0.06521202798183798

In [25]:
model.validation(categorical_X, Y, mapping_size, method=2)

0.06471905800918545